<a href="https://colab.research.google.com/github/cawllol/Deep-learning/blob/master/OptimizeCNNModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install keras-tuner

     |████████████████████████████████| 61kB 6.1MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=7df13bb8f39df6663afaa2fb8e288c97353753f6b123e1ae7ba88bd50403786a
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=dcc9496da28b026499516dd8c3b177986617aba2f8c665415f25f2432603206d
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [0]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [0]:
#importing dataset
fashion_mnist=keras.datasets.fashion_mnist

In [0]:
#making train & test images
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [0]:
#image scalling
train_images=train_images/255.0
test_images=test_images/255.0

In [0]:
train_images[0].shape

(28, 28)

In [0]:
#reshaping images
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [0]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [0]:
from kerastuner import RandomSearch #it'll search the best parameter
from kerastuner.engine.hyperparameters import HyperParameters

In [0]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=3,directory='output',project_name="Mnist Fashion")

In [0]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Epoch 1/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.5009 - accuracy: 0.8191 - val_loss: 0.4098 - val_accuracy: 0.8513
Epoch 2/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.3767 - accuracy: 0.8594 - val_loss: 0.3682 - val_accuracy: 0.8627
Epoch 3/3
1688/1688 [==============================] - 9s 5ms/step - loss: 0.3394 - accuracy: 0.8740 - val_loss: 0.4124 - val_accuracy: 0.8522


Epoch 1/3
1688/1688 [==============================] - 12s 7ms/step - loss: 0.4556 - accuracy: 0.8435 - val_loss: 0.3555 - val_accuracy: 0.8720
Epoch 2/3
1688/1688 [==============================] - 12s 7ms/step - loss: 0.3169 - accuracy: 0.8838 - val_loss: 0.3469 - val_accuracy: 0.8732
Epoch 3/3
1431/1688 [========================>.....] - ETA: 1s - loss: 0.2752 - accuracy: 0.8973

In [0]:
model=tuner_search.get_best_models(num_models=1)[0]

In [0]:
model.summary()

In [0]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)